In [1]:
import pandas as pd
import collections
import MeCab
import numpy as np
import os
import xgboost as xgb
from tqdm import tqdm

In [2]:
df_train = pd.read_excel("train.xlsx")
df_test = pd.read_excel("test.xlsx")
m = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/  -Owakati")
tqdm.pandas()

In [3]:
sorted(dict(collections.Counter(df_train["screen_name"])).items(), key = lambda w:w[1], reverse=True)

[('seeker__Taka', 2917),
 ('ss_aeaea', 2896),
 ('MinorinPart3', 2844),
 ('QGRWuB1zcxoTgJh', 2619),
 ('Yasaka_8hk', 2383),
 ('longer_u', 2256),
 ('hayao_sayuden', 2099),
 ('jukain_178', 1747),
 ('Youya_ja_nai', 1535),
 ('kumarcturus', 1274),
 ('yuhi_2018', 1229),
 ('yhis_ush', 1106),
 ('haruya_s32', 960),
 ('Nissy1234321', 703),
 ('asarimayou', 453),
 ('hd208816jp', 258),
 ('bayern1049', 258),
 ('aTwUJJhX9SSsZni', 191),
 ('solt_sato', 164),
 ('kim1014_41', 148),
 ('ce4EHOLK8UcC1RH', 123),
 ('sota_155', 96),
 ('utokyoshinkawa', 66),
 ('h8i__', 53),
 ('kuni_sawa', 27),
 ('motohar61987696', 16),
 ('ari0407ari', 3)]

In [4]:
df_test

,Unnamed: 0,id,text,reply,created_at
0,0,1138740434472198016,雨の日の水曜とかけまして、月曜１限の単位と解きます。\n\n\\\その心は///\n\nどち...,NaN,2019-06-12 09:30:51
1,1,1138269588485918976,mnys君最高すぎるな https://t.co/OjTOTcGnk6,NaN,2019-06-11 02:19:52
2,2,1137099105019105024,激ウマだった。\nアメオリエンご褒美〜\n(少人数遠征もたのしい) https://t.co...,NaN,2019-06-07 20:48:47
3,3,1136290473268530944,@JWU_KA_momona (自転車タイプを漕ぎまくったら、翌日の七国450mアップに耐え...,8.516517e+17,2019-06-05 15:15:34
4,4,1136277817832820992,@JWU_KA_momona 競技情報についてお尋ねしたいのですが、使用するボードは手で漕ぐ...,8.516517e+17,2019-06-05 14:25:17
5,5,1136267358224281984,@aerialsword 参加します,8.477109e+17,2019-06-05 13:43:43
6,6,1135450937047126016,@domitorianeast そうなんですか！\nありがとうございます！,2.522091e+09,2019-06-03 07:39:33
7,7,1135424022861176960,メーリスってたしかこれ使ってるのでは？ https://t.co/ZISdLBfE2w,NaN,2019-06-03 05:52:36
8,8,1134295070776077952,【悲報】\nmyjn、一年英語&amp;二年英語を寝ぶっち。留年へまた一歩近づく。,NaN,2019-05-31 03:06:33
9,9,1132855499412520960,お隣の部室より\n\nA（着替えながら）「OLKの人入ってきちゃうかも。かりんちゃん来ちゃう...,NaN,2019-05-27 03:46:13
